# Домашнее задание для DS по разработке PD модели

### Необходиом разработать модель предсказания дефолта. Для разрабокти модели даны финансовые данные 32 595 компаний (выручка, активы, пассивы и т.д). Оценка результатапроводится на закрытой части по 200 компаниям. метрика оценки accuracy_score

### Импортируем библиотеки

In [3]:
import pandas as pd
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

### Подгружаем данные

In [4]:
!wget https://raw.githubusercontent.com/BKHV/ds-test/master/PD-data.csv

/bin/sh: wget: command not found


In [5]:
df = pd.read_csv('PD-data.csv', sep=';')

Размеры выборок

In [6]:
print(df.shape)

(32595, 37)


Количество разных типов фичей

In [7]:
df.dtypes.value_counts()

float64    25
int64      11
object      1
dtype: int64

Уровень дефолт-рейта в выборке = 6,7219%

In [8]:
df['default_12m'].values.mean()

0.06721889860408038

In [9]:
df.record_id.nunique()

32595

32595 уникальных наблюдений

### Строим бэйзлайн модель c 3-ми фичами

In [10]:
features = ['bus_age','ul_capital_sum', 'ul_founders_cnt']

In [11]:
features_df = df.loc[:,features]
features_df['default_12m'] = df['default_12m']

In [12]:
features_df.head()

,bus_age,ul_capital_sum,ul_founders_cnt,default_12m
0,198,50000.0,2,0
1,256,10000.0,5,0
2,33,10000.0,2,0
3,185,300000.0,1,0
4,220,10000.0,2,0


### Построим карту коллеряций

In [13]:
features_df.corr()

,bus_age,ul_capital_sum,ul_founders_cnt,default_12m
bus_age,1.000000,0.067037,0.209244,-0.111336
ul_capital_sum,0.067037,1.000000,0.029621,-0.019556
ul_founders_cnt,0.209244,0.029621,1.000000,-0.024054
default_12m,-0.111336,-0.019556,-0.024054,1.000000


In [14]:
X_train, X_test, y_train, y_test = train_test_split(features_df.drop('default_12m',axis=1),
                                                    features_df['default_12m'],
                                                    random_state=42, shuffle=True)

logit = LogisticRegression(C=1,solver='lbfgs',max_iter=1000)
cv_baseline = cross_val_score(logit, X_train, y_train, cv=5, scoring='roc_auc').mean()
cv_baseline

0.6488699458962778

ROC/AUC бэйзлайн модели - 64,89%

In [19]:
logit.fit(X_train, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [20]:
answ_df = df[['record_id']]
answ_df['id'] = answ_df[['record_id']].copy()
answ_df.drop('record_id',axis=1, inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [36]:
cutoff = 0.0672
answ_df['predict'] = (logit.predict_proba(features_df.drop('default_12m',axis=1))[:,1]> cutoff).astype(int)



/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
answ_df.head()

,id,predict
0,7,0
1,13,0
2,18,1
3,22,0
4,23,0


In [38]:
answ_df.to_csv('PD-submit.csv',index=False,sep=';')

Для загрузки своего рузультата на https://dsbattle.com/hackathons/juniords/ вам нужно создать csv файл с двумя колонками (id, predict), разделитель - точка с запятой (;)